<a href="https://colab.research.google.com/github/ceydab/Python-Practice-Applications/blob/main/CNN_for_twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/cardiffnlp/tweeteval.git

fatal: destination path 'tweeteval' already exists and is not an empty directory.


In [2]:
import os
import re
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.utils.data as data

In [3]:
# use the GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [4]:
#create the dataframes


label_mapping = {}
with open("tweeteval/datasets/emotion/mapping.txt", 'r') as file:
    for line in file:
        label, emotion = line.strip().split('\t')
        label_mapping[label] = emotion


# Load and process your data
def load_and_filter_data(text_file, label_file):
    texts = []
    labels = []
    with open(text_file, 'r') as file1, open(label_file, 'r') as file2:
        while True:
            text = file1.readline().strip()
            label = file2.readline().strip()

            if not text or not label:
                break

            emotion = label_mapping.get(label, label)  # Map label to emotion using the mapping
            texts.append(text)
            labels.append(emotion)

    return texts, labels


train_df = pd.DataFrame()
val_df = pd.DataFrame()
test_df = pd.DataFrame()
train_df["text"], train_df["label"] = load_and_filter_data("tweeteval/datasets/emotion/train_text.txt", "tweeteval/datasets/emotion/train_labels.txt")
val_df["text"], val_df["label"] = load_and_filter_data("tweeteval/datasets/emotion/val_text.txt", "tweeteval/datasets/emotion/val_labels.txt")
test_df["text"], test_df["label"] = load_and_filter_data("tweeteval/datasets/emotion/test_text.txt", "tweeteval/datasets/emotion/test_labels.txt")
train_df.drop_duplicates(inplace=True)

X_train, y_train = train_df["text"], train_df["label"]
X_val, y_val= val_df["text"], val_df["label"]
X_test, y_test = test_df["text"], test_df["label"]

In [5]:
#preprocess imports
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer, sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
import string


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def preprocessing(tweet_text):
    tweet_text = tweet_text.lower()
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(tweet_text)
  # Remove stopwords, punctuation, hashtags, mentions, URLs
    tokens = [token for token in tokens if token not in stopwords.words('english') and token not in string.punctuation and not token.startswith("#") and not token.startswith("@") and not token.startswith("http")]

#emojis
    tokens = [token for token in tokens if token.isalnum() or token.isascii()]

    processed_tweet = ' '.join(tokens)
    return processed_tweet


X_train = X_train.apply(preprocessing)
X_val = X_val.apply(preprocessing)
X_test = X_test.apply(preprocessing)


In [7]:
#tokenize, vectorize, and tensor
vocab = set(X_train)
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for i, word in enumerate(vocab)}

word_to_ix['<unk>'] = len(word_to_ix)
ix_to_word[len(word_to_ix)] = '<unk>'

def tokenize_and_index(text, word_to_ix):
    tokens = text.split()
    indexed_tokens = [word_to_ix.get(token, word_to_ix['<unk>']) for token in tokens]
    return indexed_tokens

# Apply tokenization and indexing
X_train_indices = [tokenize_and_index(text, word_to_ix) for text in X_train]
X_val_indices = [tokenize_and_index(text, word_to_ix) for text in X_val]
X_test_indices = [tokenize_and_index(text, word_to_ix) for text in X_test]



X_train_tensor = [torch.LongTensor(indices) for indices in X_train_indices]
X_val_tensor = [torch.LongTensor(indices) for indices in X_val_indices]
X_test_tensor = [torch.LongTensor(indices) for indices in X_test_indices]

from torch.nn.utils.rnn import pad_sequence
X_train_tensor = pad_sequence([torch.LongTensor(indices) for indices in X_train_indices], batch_first=True)
X_val_tensor = pad_sequence([torch.LongTensor(indices) for indices in X_val_indices], batch_first=True)
X_test_tensor = pad_sequence([torch.LongTensor(indices) for indices in X_test_indices], batch_first=True)




In [8]:
print(y_train.value_counts())

anger       1400
sadness      855
joy          683
optimism     294
Name: label, dtype: int64


In [9]:
# manual labeling and encoding
label_mapping = {'anger': 0, 'sadness': 1, 'joy':2, 'optimism':3}

y_train_encoded = [label_mapping[label] for label in y_train]
y_val_encoded = [label_mapping[label] for label in y_val]
y_test_encoded = [label_mapping[label] for label in y_test]

y_train_tensor = torch.Tensor(y_train_encoded).long()
y_val_tensor = torch.Tensor(y_val_encoded).long()
y_test_tensor = torch.Tensor(y_test_encoded).long()

In [10]:
# DataLoader for sets
batch_size = 128
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [32]:
#CNN structure

class SentimentCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, filter_sizes, num_filters,  output_dim, dropout):
        super(SentimentCNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(in_channels=1, out_channels=nf, kernel_size=(fs, embedding_dim))
            for nf, fs in zip(num_filters, filter_sizes)
        ])
        self.fc = nn.Linear(len(filter_sizes)*num_filters[0], output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.unsqueeze(1)

        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        cat = self.dropout(torch.cat(pooled, dim=1))

        return self.fc(cat)



In [12]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [30]:
#get the best number of filters

import itertools
numbers = [3, 4, 5]
num_filters = [[150, 150,150], [200,200,200], [250,250,250]]
filter_sizes = [3,4,5]
# Initialize variables to keep track of the best hyperparameters and metrics
best_hyperparameters = None
best_accuracy = 0.0

# Perform grid search
for filter_size, num_filter,  in itertools.product(filter_sizes, num_filters):
    batch_size = 128
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # Create a new model with the current hyperparameters
    model = SentimentCNN(
        vocab_size=len(word_to_ix),
        embedding_dim=500,
        filter_sizes=filter_size,
        num_filters=num_filter,
        output_dim=4,
        dropout=0.2
    )

    # Define the loss and optimizer
    criterion = nn.CrossEntropyLoss(weight = torch.FloatTensor([1.0, 2.0, 2.0, 6.0]))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 10  # Adjust the number of epochs as needed

    # Training loop (similar to your existing code)
    for epoch in range(num_epochs):

      model.train()
      total_loss = 0.0

      for batch in train_loader:
          inputs, labels = batch
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      # Calculate and print the average loss for this epoch
      avg_loss = total_loss / len(train_loader)
      print(f'Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_loss:.4f}')

      # Validation
      model.eval() #removes dropout
      val_loss = 0.0
      correct = 0
      total = 0
      all_predictions = []
      with torch.no_grad():
          for batch in val_loader:
              inputs, labels = batch
              outputs = model(inputs)
              loss = criterion(outputs, labels)
              val_loss += loss.item()
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()
              all_predictions.extend(predicted.tolist())
      val_accuracy = 100 * correct / total
      val_loss /= len(val_loader)

      print(f'Epoch [{epoch+1}/{num_epochs}] - Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')


    # Calculate accuracy and F1-macro scores
    accuracy = accuracy_score(y_val_encoded, all_predictions)

    # Keep track of the best hyperparameters
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = (num_filter, filter_size)

# Print the best hyperparameters and their corresponding accuracy
print("Best Hyperparameters:")
print("Number of filters:", best_hyperparameters[0])
print("Filter size:", best_hyperparameters[1])
print("Best Accuracy:", best_accuracy)

torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([32, 450])
Epoch [1/10] - Train Loss: 1.8774
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([118, 450])
Epoch [1/10] - Validation Loss: 1.3835, Accuracy: 15.78%
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([128, 450])
torch.Size([1

In [ ]:
#get the best filter size
numbers = [3, 4, 5]
filter_sizes = [list(combination) for combination in itertools.product(numbers, repeat=3)]
# Initialize variables to keep track of the best hyperparameters and metrics
best_hyperparameters = None
best_accuracy = 0.0

# Perform grid search
for filter_size, in itertools.product(filter_sizes):
    batch_size = 128
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # Create a new model with the current hyperparameters
    model = SentimentCNN(
        vocab_size=len(word_to_ix),
        embedding_dim=500,
        filter_sizes=filter_size,
        num_filters=[200,200,200],
        output_dim=4,
        dropout=0.2
    )

    # Define the loss and optimizer
    criterion = nn.CrossEntropyLoss(weight = torch.FloatTensor([1.0, 2.0, 2.0, 6.0]))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    num_epochs = 10  # Adjust the number of epochs as needed

    # Training loop (similar to your existing code)
    for epoch in range(num_epochs):

      model.train()
      total_loss = 0.0

      for batch in train_loader:
          inputs, labels = batch
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      # Calculate and print the average loss for this epoch
      avg_loss = total_loss / len(train_loader)
      print(f'Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_loss:.4f}')

      # Validation
      model.eval() #removes dropout
      val_loss = 0.0
      correct = 0
      total = 0
      all_predictions = []
      with torch.no_grad():
          for batch in val_loader:
              inputs, labels = batch
              outputs = model(inputs)
              loss = criterion(outputs, labels)
              val_loss += loss.item()
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()
              all_predictions.extend(predicted.tolist())
      val_accuracy = 100 * correct / total
      val_loss /= len(val_loader)

      print(f'Epoch [{epoch+1}/{num_epochs}] - Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')


    # Calculate accuracy and F1-macro scores
    accuracy = accuracy_score(y_val_encoded, all_predictions)

    # Keep track of the best hyperparameters
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = (filter_size)

# Print the best hyperparameters and their corresponding accuracy
print("Best Hyperparameters:")
print("Filter size:", best_hyperparameters[0])
print("Best Accuracy:", best_accuracy)

Epoch [1/10] - Train Loss: 1.8170
Epoch [1/10] - Validation Loss: 1.3673, Accuracy: 20.32%
Epoch [2/10] - Train Loss: 1.3670
Epoch [2/10] - Validation Loss: 1.4447, Accuracy: 15.78%
Epoch [3/10] - Train Loss: 1.3434
Epoch [3/10] - Validation Loss: 1.3651, Accuracy: 19.52%
Epoch [4/10] - Train Loss: 1.3066
Epoch [4/10] - Validation Loss: 1.3427, Accuracy: 28.61%
Epoch [5/10] - Train Loss: 1.2816
Epoch [5/10] - Validation Loss: 1.3350, Accuracy: 21.12%
Epoch [6/10] - Train Loss: 1.2609
Epoch [6/10] - Validation Loss: 1.3199, Accuracy: 22.73%
Epoch [7/10] - Train Loss: 1.2494
Epoch [7/10] - Validation Loss: 1.3198, Accuracy: 30.48%
Epoch [8/10] - Train Loss: 1.2401
Epoch [8/10] - Validation Loss: 1.3285, Accuracy: 25.67%
Epoch [9/10] - Train Loss: 1.2388
Epoch [9/10] - Validation Loss: 1.3051, Accuracy: 25.13%
Epoch [10/10] - Train Loss: 1.2438
Epoch [10/10] - Validation Loss: 1.3048, Accuracy: 27.01%
Epoch [1/10] - Train Loss: 2.0329
Epoch [1/10] - Validation Loss: 1.4254, Accuracy: 24.0

In [ ]:
#rerun with the obtained best filters to get the best learning rate, dropout rate, batch size, optimizer, embedding dim, epoch number
learning_rates = [0.001, 0.01, 0.0001]
dropout_rates = [0.2, 0.5, 0.8]
batch_sizes = [128,64]
optimizers = [torch.optim.Adam, torch.optim.SGD]
embedding_dims = [500,300]
epochs = [13, 18]

# Initialize variables to keep track of the best hyperparameters and metrics
best_hyperparameters = None
best_accuracy = 0.0

# Perform grid search
for learning_rate, dropout_rate, batch_size, optimizer, embedding_dim, epoch  in itertools.product(learning_rates, dropout_rates, batch_sizes, optimizers, embedding_dims, epochs):
    batch_size = batch_size
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    # Create a new model with the current hyperparameters
    model = SentimentCNN(
        vocab_size=len(word_to_ix),
        embedding_dim=embedding_dim,
        filter_sizes=[,,],
        num_filters=[200,200,200],
        output_dim=4,
        dropout=dropout_rate
    )

    # Define the loss and optimizer
    criterion = nn.CrossEntropyLoss(weight = torch.FloatTensor([1.0, 2.0, 2.0, 6.0]))
    optimizer = optimizer(model.parameters(), lr=learning_rate)

    num_epochs = epoch  # Adjust the number of epochs as needed

    # Training loop (similar to your existing code)
    for epoch in range(num_epochs):

      model.train()
      total_loss = 0.0

      for batch in train_loader:
          inputs, labels = batch
          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
          total_loss += loss.item()

      # Calculate and print the average loss for this epoch
      avg_loss = total_loss / len(train_loader)
      print(f'Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_loss:.4f}')

      # Validation
      model.eval() #removes dropout
      val_loss = 0.0
      correct = 0
      total = 0
      all_predictions = []
      with torch.no_grad():
          for batch in val_loader:
              inputs, labels = batch
              outputs = model(inputs)
              loss = criterion(outputs, labels)
              val_loss += loss.item()
              _, predicted = torch.max(outputs.data, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()
              all_predictions.extend(predicted.tolist())
      val_accuracy = 100 * correct / total
      val_loss /= len(val_loader)

      print(f'Epoch [{epoch+1}/{num_epochs}] - Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')


    # Calculate accuracy and F1-macro scores
    accuracy = accuracy_score(y_val_encoded, all_predictions)

    # Keep track of the best hyperparameters
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_hyperparameters = (learning_rate, dropout_rate, batch_size, optimizer, embedding_dim, epoch)

# Print the best hyperparameters and their corresponding accuracy
print("Best Hyperparameters:")
print("Learning Rate:", best_hyperparameters[0])
print("Dropout Rate:", best_hyperparameters[1])
print("Batch Size:", best_hyperparameters[2])
print("Optimizer:", best_hyperparameters[3])
print("Embedding Dimensions:", best_hyperparameters[4])
print("Epochs:", best_hyperparameters[5])
print("Best Accuracy:", best_accuracy)

In [20]:
# using the best hyperparameters obtained from the iteration above, substitute the hyperparameters for the best model
model = SentimentCNN(
                     vocab_size=len(word_to_ix),
                     embedding_dim= 500,
                     filter_sizes=[3, 4, 5],
                     num_filters= [300, 350, 400],
                     output_dim=4,
                     dropout=0.2)




criterion = nn.CrossEntropyLoss(weight = torch.FloatTensor([1.0, 2.0, 2.0, 6.0]))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [21]:
#training and validation loops for the best model


num_epochs = 15
torch.manual_seed(0)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Calculate and print the average loss for this epoch
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_loss:.4f}')

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    predictionsval = []
    labelsval = []

    with torch.no_grad():
        for batch in val_loader:
            inputs, labels = batch
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            predictionsval.extend(predicted.tolist())
            labelsval.extend(labels.tolist())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    val_loss /= len(val_loader)

    print(f'Epoch [{epoch+1}/{num_epochs}] - Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')



torch.Size([128, 1050])


RuntimeError: ignored

In [37]:
print(correct, "/", total)

#batch128=172, batch64= 166 batch256=114                            ----> batch128
#lr0.001 =172, lr0.01 =76, lr0.0001 = 109                           ----> lr0.001
#embed300=172, embed500=180, embed200=108, embed600=109             ---->embed500
#drop0.5=177, drop0.2= 180, drop0.8= 102                            ---->drop0.2

175 / 374


In [30]:

# Calculate accuracy and F1-macro scores for validation set
accuracy = accuracy_score(labelsval, predictionsval)
f1_macro = f1_score(labelsval, predictionsval, average='macro')

# Create a classification report for detailed metrics
report = classification_report(labelsval, predictionsval, target_names=['Anger', 'Sadness', 'Joy', 'Optimism'], output_dict=True)

# Create a table to display the results
results = pd.DataFrame({
    'Metric': ['Accuracy', 'F1-Macro'],
    'Value': [accuracy, f1_macro]
})

# Display the table
print(results)

# Display the detailed classification report
print(pd.DataFrame(report).transpose())

     Metric     Value
0  Accuracy  0.470588
1  F1-Macro  0.336472
              precision    recall  f1-score     support
Anger          0.491736  0.743750  0.592040  160.000000
Sadness        0.566667  0.191011  0.285714   89.000000
Joy            0.438202  0.402062  0.419355   97.000000
Optimism       0.076923  0.035714  0.048780   28.000000
accuracy       0.470588  0.470588  0.470588    0.470588
macro avg      0.393382  0.343134  0.336472  374.000000
weighted avg   0.464627  0.470588  0.433685  374.000000


In [17]:

torch.save(model.state_dict(), 'sentiment_cnn_model.pth')


In [18]:
model.load_state_dict(torch.load('sentiment_cnn_model.pth'))
model.eval()

# Initialize lists to store predictions and true labels
all_predictions = []
all_true_labels = []

# Iterate through the test data
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        all_predictions.extend(predicted.tolist())
        all_true_labels.extend(labels.tolist())

# Convert the predictions and true labels to numpy arrays
predictions = np.array(all_predictions)
true_labels = np.array(all_true_labels)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Calculate accuracy and F1-macro scores for test set
accuracy = accuracy_score(true_labels, predictions)
f1_macro = f1_score(true_labels, predictions, average='macro')

# Create a classification report for detailed metrics
report = classification_report(true_labels, predictions, target_names=['Anger', 'Sadness', 'Joy', 'Optimism'], output_dict=True)

# Create a table to display the results
results = pd.DataFrame({
    'Metric': ['Accuracy', 'F1-Macro'],
    'Value': [accuracy, f1_macro]
})

# Display the table
print(results)

# Display the detailed classification report
print(pd.DataFrame(report).transpose())


     Metric     Value
0  Accuracy  0.438424
1  F1-Macro  0.346878
              precision    recall  f1-score      support
Anger          0.487085  0.709677  0.577681   558.000000
Sadness        0.479554  0.337696  0.396313   382.000000
Joy            0.441489  0.231844  0.304029   358.000000
Optimism       0.099338  0.121951  0.109489   123.000000
accuracy       0.438424  0.438424  0.438424     0.438424
macro avg      0.376866  0.350292  0.346878  1421.000000
weighted avg   0.440010  0.438424  0.419456  1421.000000
